In [1]:
import pandas as pd
import numpy as np

# Importing Data

In [2]:
df_main = pd.read_csv('CSV Files/main.csv')

In [3]:
df_train = pd.read_csv('CSV Files/train.csv')

In [4]:
df_init = df_main.drop(['Stars'], axis = 1)
df_train = df_train[['Index', 'User_id', 'Product_id', 'Stars']]
df_train = pd.merge(df_init, df_train, how = 'left', on = ['Index', 'User_id', 'Product_id'])
df_train.Stars = df_train.Stars.fillna(0)
df_train

,Index,User_id,Product_id,Username,Review,Review_cleaned,Combination,Dry,Normal,Oily,Sensitive,Skin_Tone,Skin_Type,Eye_Color,Hair_Color,category,price,Stars
0,1,3420,157.0,allyp3,This is hands down the best cleanser I've ever...,"['hand', 'best', 'cleanser', 'ever', 'used', '...",0,0,0,0,0,Medium,Combination,Brown,Brunette,Cleanser,36.00,5.0
1,2,2483,157.0,PatTea,Unfortunately this doesn't work for everyone. ...,"['unfortunately', 'work', 'everyone', 'used', ...",0,0,0,0,0,Medium,Combination,Brown,Red,Cleanser,36.00,1.0
2,3,2715,157.0,Sabi1991,My favorite cleanser!! i love the packaging on...,"['favorite', 'cleanser', 'love', 'packaging', ...",0,0,0,0,0,No data,No data,No data,No data,Cleanser,36.00,5.0
3,4,4497,157.0,happyface2,I love all things Youth To The People! This cl...,"['love', 'thing', 'youth', 'people', 'cleanser...",0,0,0,0,0,Fair,Dry,Blue,Blonde,Cleanser,36.00,5.0
4,5,5017,157.0,kimkix34,I had a trial size of this and was obsessed. M...,"['trial', 'size', 'wa', 'obsessed', 'skin', 'f...",0,0,0,0,0,Fair,Normal,Green,Blonde,Cleanser,36.00,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,13127,10169,1440.0,SUSANNA,"It's great, love it. It makes skin so soft, sm...",NaN,0,0,0,0,0,No data,No data,No data,No data,Bath Oil,8.80,5.0
13127,13128,10353,1443.0,Michi,I am struggling with bowel cancer and the writ...,NaN,0,0,0,0,0,No data,No data,No data,No data,Bath Oil,49.00,5.0
13128,13129,10354,1452.0,Siii,"Good product, smells nice. Will be good for ou...",NaN,0,0,0,0,0,No data,No data,No data,No data,Bath Oil,13.95,5.0
13129,13130,8312,1452.0,nina,This Weleda Calendula Cream Bath is so soft an...,NaN,0,0,0,0,0,No data,No data,No data,No data,Bath Oil,13.95,4.0


# Data Preprocessing

## Create User User Dataframe

In [5]:
user_user_df = df_train.drop(columns = ['Index'])

In [6]:
user_user_df.shape

(13131, 17)

## One hot encoding for Properties into 1 column

In [7]:
user_properties = user_user_df[['User_id','Skin_Tone','Skin_Type','Eye_Color','Hair_Color']]

In [8]:
# Need to run through the user properties because some users indicated different data for different entries
# decided to keep anything that is not 'No data' if there are duplicates
cleansed_user_properties = []
user_list = np.sort(user_properties.User_id.unique())
counter = 0

for i in user_list:
    holder = user_properties[user_properties.User_id == i]
    if len(holder.User_id) == 1:
        Skin_Tone = holder.Skin_Tone.to_list()[0]
        Skin_Type = holder.Skin_Type.to_list()[0]
        Eye_Color = holder.Eye_Color.to_list()[0]
        Hair_Color = holder.Hair_Color.to_list()[0]
        counter += 1
    else:
        for j, row in enumerate(holder.itertuples()):
            if j == 0:
                Skin_Tone = row.Skin_Tone
                Skin_Type = row.Skin_Type
                Eye_Color = row.Eye_Color
                Hair_Color = row.Hair_Color
            else:
                Skin_Tone2 = row.Skin_Tone
                Skin_Type2 = row.Skin_Type
                Eye_Color2 = row.Eye_Color
                Hair_Color2 = row.Hair_Color
                if Skin_Tone2 != Skin_Tone:
                    if Skin_Tone2 != 'No data':
                        Skin_Tone = Skin_Tone2
                if Skin_Type2 != Skin_Type:
                    if Skin_Type2 != 'No data':
                        Skin_Type = Skin_Type2
                if Eye_Color2 != Eye_Color:
                    if Eye_Color2 != 'No data':
                        Eye_Color = Eye_Color2
                if Hair_Color2 != Hair_Color:
                    if Hair_Color2 != 'No data':
                        Hair_Color = Hair_Color2
        
    holding_list = [i, Skin_Tone, Skin_Type, Eye_Color, Hair_Color]          
    cleansed_user_properties.append(holding_list)
    
cleansed_user_properties_df = pd.DataFrame (cleansed_user_properties, 
                                            columns = ['User_id', 'Skin_Tone', 'Skin_Type', 'Eye_Color', 'Hair_Color',])

In [9]:
categorical_cols = ['Skin_Tone','Skin_Type','Eye_Color','Hair_Color']
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
cleansed_user_properties_df[categorical_cols] = cleansed_user_properties_df[categorical_cols].apply(lambda col: le.fit_transform(col))    
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [10]:
_, new_cols = np.unique(cleansed_user_properties_df[['Skin_Tone','Skin_Type','Eye_Color','Hair_Color']].to_numpy(), axis =0, return_inverse=True)
cleansed_user_properties_df['Combine_Properties'] = new_cols

# Recommendation Algorithm

## Provide recommendation based on Combine Properties

In [11]:
def Recommendation_w_properties_only(selected_id, num_of_products):
    users = []
    items = []
    ratings = []
    priority = []
    
    
    # Selected User 
    selected_skin_tone = cleansed_user_properties_df[
        cleansed_user_properties_df.User_id == selected_id].Skin_Tone.to_list()[0]
    selected_skin_type = cleansed_user_properties_df[
        cleansed_user_properties_df.User_id == selected_id].Skin_Type.to_list()[0]
    selected_eye_color = cleansed_user_properties_df[
        cleansed_user_properties_df.User_id == selected_id].Eye_Color.to_list()[0]
    selected_hair_color = cleansed_user_properties_df[
        cleansed_user_properties_df.User_id == selected_id].Hair_Color.to_list()[0]
    
    # Find users who have 4 same properties as the Selected User
    same_users_list_4 = cleansed_user_properties_df[(cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
                                                 (cleansed_user_properties_df.Skin_Type == selected_skin_type) &
                                                 (cleansed_user_properties_df.Eye_Color == selected_eye_color) &
                                                 (cleansed_user_properties_df.Hair_Color == selected_hair_color)
                                                 ].User_id.to_list()
    
    # Drop the Selected User
    same_users_list_4.remove(selected_id)
    leng = len(same_users_list_4)
    
    for i in range(leng):
        user = same_users_list_4[i]
        item_list = user_user_df[user_user_df.User_id == i].Product_id.values.tolist()
        rating_list = user_user_df[user_user_df.User_id == i].Stars.values.tolist()
        for j in range(len(item_list)):
            item = item_list[j]
            rating = rating_list[j]
            items.append(item)
            ratings.append(rating)
            users.append(user)
            priority.append(4)
            
    if len(items) < num_of_products:
        
        # Next Set of Users
        same_users_list_3a = cleansed_user_properties_df[
            (cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
            (cleansed_user_properties_df.Skin_Type == selected_skin_type) &
            (cleansed_user_properties_df.Eye_Color == selected_eye_color)
        ].User_id.to_list()

        same_users_list_3b = cleansed_user_properties_df[
            (cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
            (cleansed_user_properties_df.Eye_Color == selected_eye_color) &
            (cleansed_user_properties_df.Hair_Color == selected_hair_color)
        ].User_id.to_list()

        same_users_list_3c = cleansed_user_properties_df[
            (cleansed_user_properties_df.Skin_Type == selected_skin_type) &
            (cleansed_user_properties_df.Eye_Color == selected_eye_color) &
            (cleansed_user_properties_df.Hair_Color == selected_hair_color)
        ].User_id.to_list()

        same_users_list_3d = cleansed_user_properties_df[
            (cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
            (cleansed_user_properties_df.Skin_Type == selected_skin_type) &
            (cleansed_user_properties_df.Hair_Color == selected_hair_color)
        ].User_id.to_list()
        
        list1 = [x for x in same_users_list_3a if x not in same_users_list_4]
        list2 = [x for x in same_users_list_3b if x not in same_users_list_4]
        list3 = [x for x in same_users_list_3c if x not in same_users_list_4]
        list4 = [x for x in same_users_list_3d if x not in same_users_list_4]
        combined_list = list1 + list2 + list3 + list4
        final_list_3 = []
        for i in combined_list:
            if i not in final_list_3:
                final_list_3.append(i)
        
        similar_users_found = final_list_3 + same_users_list_4
        
        for i in final_list_3:
            user = i
            item_list = user_user_df[user_user_df.User_id == i].Product_id.values.tolist()
            rating_list = user_user_df[user_user_df.User_id == i].Stars.values.tolist()
            for j in range(len(item_list)):
                item = item_list[j]
                rating = rating_list[j]
                items.append(item)
                ratings.append(rating)
                users.append(user)
                priority.append(3)
        
        if len(items) < num_of_products:
            
            # Next Set of Users
            same_users_list_2a = cleansed_user_properties_df[
                (cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
                (cleansed_user_properties_df.Skin_Type == selected_skin_type)
            ].User_id.to_list()
            
            same_users_list_2b = cleansed_user_properties_df[
                (cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
                (cleansed_user_properties_df.Hair_Color == selected_hair_color)
            ].User_id.to_list()
            
            same_users_list_2c = cleansed_user_properties_df[
                (cleansed_user_properties_df.Skin_Tone == selected_skin_tone) &
                (cleansed_user_properties_df.Eye_Color == selected_eye_color)
            ].User_id.to_list()
            
            
            same_users_list_2d = cleansed_user_properties_df[
                (cleansed_user_properties_df.Skin_Type == selected_skin_type) &
                (cleansed_user_properties_df.Eye_Color == selected_eye_color)
            ].User_id.to_list()
            
            
            same_users_list_2e = cleansed_user_properties_df[
                (cleansed_user_properties_df.Skin_Type == selected_skin_type) &
                (cleansed_user_properties_df.Hair_Color == selected_hair_color)
            ].User_id.to_list()
            
            
            same_users_list_2f = cleansed_user_properties_df[
                (cleansed_user_properties_df.Eye_Color == selected_eye_color) &
                (cleansed_user_properties_df.Hair_Color == selected_hair_color)
            ].User_id.to_list()
            
            
            list1 = [x for x in same_users_list_2a if x not in similar_users_found]
            list2 = [x for x in same_users_list_2b if x not in similar_users_found]
            list3 = [x for x in same_users_list_2c if x not in similar_users_found]
            list4 = [x for x in same_users_list_2d if x not in similar_users_found]
            list5 = [x for x in same_users_list_2e if x not in similar_users_found]
            list6 = [x for x in same_users_list_2f if x not in similar_users_found]
            
            combined_list = list1 + list2 + list3 + list4 + list5 + list6
            final_list_2 = []
            for i in combined_list:
                if i not in final_list_2:
                    final_list_2.append(i)
                
            similar_users_found = similar_users_found + final_list_2
            
            for i in final_list_2:
                user = i
                item_list = user_user_df[user_user_df.User_id == i].Product_id.values.tolist()
                rating_list = user_user_df[user_user_df.User_id == i].Stars.values.tolist()
                for j in range(len(item_list)):
                    item = item_list[j]
                    rating = rating_list[j]
                    items.append(item)
                    ratings.append(rating)
                    users.append(user)
                    priority.append(2)
                
            if len(items) < num_of_products:
            
                # Next Set of Users
                same_users_list_1a = cleansed_user_properties_df[
                    (cleansed_user_properties_df.Skin_Tone == selected_skin_tone)
                                                                ].User_id.to_list()

                same_users_list_1b = cleansed_user_properties_df[
                    (cleansed_user_properties_df.Hair_Color == selected_hair_color)
                                                                ].User_id.to_list()

                same_users_list_1c = cleansed_user_properties_df[
                    (cleansed_user_properties_df.Eye_Color == selected_eye_color)
                                                                ].User_id.to_list()


                same_users_list_1d = cleansed_user_properties_df[
                    (cleansed_user_properties_df.Skin_Type == selected_skin_type)
                                                                ].User_id.to_list()


                list1 = [x for x in same_users_list_1a if x not in similar_users_found]
                list2 = [x for x in same_users_list_1b if x not in similar_users_found]
                list3 = [x for x in same_users_list_1c if x not in similar_users_found]
                list4 = [x for x in same_users_list_1d if x not in similar_users_found]

                combined_list = list1 + list2 + list3 + list4
                final_list_1 = []
                for i in combined_list:
                    if i not in final_list_1:
                        final_list_1.append(i)

                similar_users_found = similar_users_found + final_list_1

                for i in final_list_1:
                    user = i
                    item_list = user_user_df[user_user_df.User_id == i].Product_id.values.tolist()
                    rating_list = user_user_df[user_user_df.User_id == i].Stars.values.tolist()
                    for j in range(len(item_list)):
                        item = item_list[j]
                        rating = rating_list[j]
                        items.append(item)
                        ratings.append(rating)
                        users.append(user)
                        priority.append(1)
    
    data_tuples = list(zip(items,ratings,users,priority))
    df = pd.DataFrame(data_tuples, columns=['Product_id','Stars','Users_id','Priority'])
    sorted_df = df.sort_values(by=['Priority', 'Stars'], ascending=False).drop_duplicates(subset=['Product_id'])
    Recommended_list = sorted_df.Product_id[0:num_of_products]
    return Recommended_list

In [12]:
a = Recommendation_w_properties_only(110,1000)

# Evaluation

## Outputting Evaluation Dataframe

In [13]:
df_test=pd.read_csv('CSV Files/test.csv')

In [14]:
train_set_userids = set(df_train['User_id'].unique())
test_set_userids = set(df_test['User_id'].unique())
matching_users = train_set_userids.intersection(test_set_userids) 

In [15]:
user_list = pd.DataFrame(matching_users,columns=['User_id'])
user_list

,User_id
0,8193
1,2051
2,8196
3,6154
4,6155
...,...
502,8172
503,4081
504,6134
505,8182


In [16]:
user_list = pd.read_csv('user_list.csv').User_id.unique()
user_list

array([ 8193,  2051,  8196,  6154,  6155,  6159,  8207,  8213,  4119,
        4120,  8223,  8224,  4129,  2042,  8234,  6194,  8261,  6215,
        6217,  8271,    81,    86,    88,    90,  2144,  4193,  8288,
        8294,  8295,   104,  8301,  6255,  8303,  4210,  8312,  8319,
        4227,  8329,  2195,   169,  8365,  4279,  2233,  8385,  2243,
        6353,   213,   230,  2282,  6378,  8430,  4336,  8441,   265,
        8474,  6427,  8475,  4385,  6433,  6435,  8485,   295,  8500,
        2365,  8513,  4421,  4430,   341,  2392,  8543,   354,  4458,
        2414,   368,  8566,  8568,  4485,   393,  8585,  8590,  4497,
        4501,  2458,  8611,  6579,  8627,   440,  6601,  4558,   465,
        2513,  4562,  8660,  8664,  2527,  8671,  8673,  6633,  4587,
        4588,   496,  6644,  8695,  2561,  4610,   524,  8717,  4630,
        8730,  8731,   544,  8738,   552,   553,  8750,  6707,  4673,
        8785,  2654,  8800,  2661,   614,  2670,  6777,  2685,  4742,
        8842,  8844,

In [17]:
num_of_products = 10
reco_products_user = []
counter = 0
for i in user_list:
    reco_products = Recommendation_w_properties_only(i,num_of_products).to_list()
    reco_products_user.append(reco_products)
    counter += 1
    print("Remaining number of users is", len(user_list) - counter)
    
reco_tuples = list(zip(user_list,reco_products_user))
Ten_Recommendation_Each_User = pd.DataFrame(reco_tuples, columns=['User_id','Product_ids'])

Remaining number of users is 506
Remaining number of users is 505
Remaining number of users is 504
Remaining number of users is 503
Remaining number of users is 502
Remaining number of users is 501
Remaining number of users is 500
Remaining number of users is 499
Remaining number of users is 498
Remaining number of users is 497
Remaining number of users is 496
Remaining number of users is 495
Remaining number of users is 494
Remaining number of users is 493
Remaining number of users is 492
Remaining number of users is 491
Remaining number of users is 490
Remaining number of users is 489
Remaining number of users is 488
Remaining number of users is 487
Remaining number of users is 486
Remaining number of users is 485
Remaining number of users is 484
Remaining number of users is 483
Remaining number of users is 482
Remaining number of users is 481
Remaining number of users is 480
Remaining number of users is 479
Remaining number of users is 478
Remaining number of users is 477
Remaining 

Remaining number of users is 255
Remaining number of users is 254
Remaining number of users is 253
Remaining number of users is 252
Remaining number of users is 251
Remaining number of users is 250
Remaining number of users is 249
Remaining number of users is 248
Remaining number of users is 247
Remaining number of users is 246
Remaining number of users is 245
Remaining number of users is 244
Remaining number of users is 243
Remaining number of users is 242
Remaining number of users is 241
Remaining number of users is 240
Remaining number of users is 239
Remaining number of users is 238
Remaining number of users is 237
Remaining number of users is 236
Remaining number of users is 235
Remaining number of users is 234
Remaining number of users is 233
Remaining number of users is 232
Remaining number of users is 231
Remaining number of users is 230
Remaining number of users is 229
Remaining number of users is 228
Remaining number of users is 227
Remaining number of users is 226
Remaining 

Remaining number of users is 1
Remaining number of users is 0


In [18]:
predictions_user_user = Ten_Recommendation_Each_User.Product_ids

In [19]:
import math
def indicators_5(rankedList, testList):
    
    Hits_i = 0
    #Len_R = 0
    Len_T = len(testList) # maximum number of unique users
    #MRR_i = 0
    HR_i = 0
    HR_i2 = 0
    NDCG_i = 0    
    count = 0

    for i in range(len(testList)):
        for j in testList[i]:
            count = count + 1 #number of users+product combination
    
    for i in range(len(rankedList)):
        for j in range(len(rankedList[i])):
            for k in range(len(testList[i])):
                if testList[i][k]==rankedList[i][j]:
                    Hits_i+=1
                    HR_i+=1
                    HR_i2+=1
                    #MRR_i+=1/(j+1)   
                    NDCG_i+=1/(math.log2(1+j+1))
                    break
    HR_i/=Len_T #hit rate over total number of unique users
    HR_i2/=count #hit rate over total number of users+product combination
    if Hits_i == 0:
        NDCG_i = 0
    else:
        NDCG_i/=Hits_i #Dividing by Hits_i is a common way to normalize the NDCG score, and it can be useful when we don't know the ideal ranking (i.e., we don't know the IDCG). In this case, we assume that the ideal ranking contains all relevant items in the test set, and we normalize the NDCG score by the number of relevant items in the test set (i.e., Hits_i).
    print(Hits_i)
    print(f'HR_method1={HR_i}')
    print(f'HR_method2={HR_i2}')
    print(f'NDCG={NDCG_i}')

In [20]:
test_list = []
for i in test_set_userids:
    test_list.append(df_test[df_test['User_id'] == i]['Product_id'].tolist())

actual_test = pd.DataFrame(matching_users,columns=['user_id'])
actual_test['bought'] = test_list
actual_test

,user_id,bought
0,8193,[1362.0]
1,2051,[32.0]
2,8196,[1079.0]
3,6154,[233.0]
4,6155,[55.0]
...,...,...
502,8172,[1251.0]
503,4081,[73.0]
504,6134,[154.0]
505,8182,"[972.0, 1227.0]"


In [21]:
indicators_5(predictions_user_user,test_list)

8
HR_method1=0.015779092702169626
HR_method2=0.0137221269296741
NDCG=0.36938553105651106
